In [12]:
import json
import time
import websocket
import ssl
import socks  # Provided by PySocks
import socket
from event import Event

def fetch_nostr_events(url, network, start, stop):
    # Generate a random subscription ID
    events = []
    sub_id = "" + str(int(time.time()))

    # Filter to get all events between start and stop
    req = [
        "REQ",
        sub_id,
        {
            "kinds": [30166],
            "authors": ["4e793514665a98176204243861dd3f0d9225fc23fc1b8e53a6629a4277237155"],
            "since": int(start),
            "until": int(stop)
        }
    ]

    close_req = ["CLOSE", sub_id]

    # Configure proxy if using Tor
    if network.lower() == "tor":
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
        socket.socket = socks.socksocket
        sslopt = {"cert_reqs": ssl.CERT_NONE}
    else:
        sslopt = {"cert_reqs": ssl.CERT_REQUIRED}

    def on_message(ws, message):
        print("Message received:", message)
        data = json.loads(message)
        if data[0] == "EVENT":
            try:
                Event.from_dict(data[2])
            except Exception as e:
                print(e, data[2])
            # print("Event:", data[2])
            events.append(data[2])
        elif data[0] == "EOSE":
            print("End of stored events.")
            ws.send(json.dumps(close_req))
            ws.close()

    def on_error(ws, error):
        print("Error:", error)

    def on_close(ws, close_status_code, close_msg):
        print("Connection closed.")


    def on_open(ws):
        ws.send(json.dumps(req))

    print(f"Connecting to {url} over {network}...")
    ws = websocket.WebSocketApp(
        url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )

    ws.run_forever(sslopt=sslopt)
    print("Connection closed.")
    print(f"Fetched {len(events)} events.")
    return events

# Example usage:
events = fetch_nostr_events('wss://relay.damus.io', "clearnet", 0, int(time.time()))
events

Connecting to wss://relay.damus.io over clearnet...
Message received: ["EVENT","1747660148",{"content":"{}","created_at":1747659038,"id":"1e44c999b1a1ccd01e0a6f5b31837ba634ef751448d70dc40d5ea0dae9535085","kind":30166,"pubkey":"4e793514665a98176204243861dd3f0d9225fc23fc1b8e53a6629a4277237155","sig":"3ff1396b95d6ca30a51bb14eb70950acbaceb81a25e466dfe1528b3b1324de0b91df6f1cfe20f5dd83dd8aa2c0d4bf7d1d48d63eb8d42b90202167884511b827","tags":[["d","wss://relay.damus.io"]]}]
Message received: ["EVENT","1747660148",{"content":"{}","created_at":1747596005,"id":"90237b055983718299bb55267c8147dbed219d151c4e772562aa2c5391994b79","kind":30166,"pubkey":"4e793514665a98176204243861dd3f0d9225fc23fc1b8e53a6629a4277237155","sig":"cd78c2cc54c4693830b3315652346af2dba99dc889b95c8c66447ef15f0afc1c8406aaf3f86296cef9d197ee88d3edc1b415cb6efed3b7454d3cd9d30bbc50b4","tags":[["d","wss://aaa-api.freefrom.space/v1/ws"]]}]
Message received: ["EVENT","1747660148",{"content":"{}","created_at":1747595830,"id":"b003c937e056

[{'content': '{}',
  'created_at': 1747659038,
  'id': '1e44c999b1a1ccd01e0a6f5b31837ba634ef751448d70dc40d5ea0dae9535085',
  'kind': 30166,
  'pubkey': '4e793514665a98176204243861dd3f0d9225fc23fc1b8e53a6629a4277237155',
  'sig': '3ff1396b95d6ca30a51bb14eb70950acbaceb81a25e466dfe1528b3b1324de0b91df6f1cfe20f5dd83dd8aa2c0d4bf7d1d48d63eb8d42b90202167884511b827',
  'tags': [['d', 'wss://relay.damus.io']]},
 {'content': '{}',
  'created_at': 1747596005,
  'id': '90237b055983718299bb55267c8147dbed219d151c4e772562aa2c5391994b79',
  'kind': 30166,
  'pubkey': '4e793514665a98176204243861dd3f0d9225fc23fc1b8e53a6629a4277237155',
  'sig': 'cd78c2cc54c4693830b3315652346af2dba99dc889b95c8c66447ef15f0afc1c8406aaf3f86296cef9d197ee88d3edc1b415cb6efed3b7454d3cd9d30bbc50b4',
  'tags': [['d', 'wss://aaa-api.freefrom.space/v1/ws']]},
 {'content': '{}',
  'created_at': 1747595830,
  'id': 'b003c937e056ae58afed9ad734378bc2c09f038d7426a896c527f8575b283cdf',
  'kind': 30166,
  'pubkey': '4e793514665a98176204243